1. $m_w$ 추가 발사대에 적재된 유도탄 수량, M을 W로 나눈 뒤 남은 값은 w_1 부터 차레로 적재

---

# 230823
제약 조건 (4), (5) 구현 시도

- 한 발사대가 연속발사를 하기 위해서는 발사절차 및 발사대 안정화를 위한 시간을 고려하여야 하므로, 수식 (4)과 같이 한 발사대 상의 연속적 발사를 위한 최소 발사 간격을 제약으로 고려한다.
- 각 발사대에서 발사된 유도탄은 제한된 공간 영역 상에서 표적을 요격할 수 있다. 표적의 예상 궤적과 발사대 위치에 따른 요격 공간으로부터 교전 가능한 유도탄 발사 시간이 계산되어진다. 위협 표적에 대해서 교전 가능한 발사 시간 구간 내에서 발사 시점이 산출되어야만 교전 가능하므로, 수식 (5)와 같이 표적 $t$에 대한 발사대 $w$에서 발사 가능한 시간에 대한 제약 조건을 고려해야 한다.

- 문제는 요격탄 발사 간격이 1초일 경우, 제한된 시간이 10초 이내 라면, 10개 이상을 발사할 수가 없을텐데,

In [56]:
import numpy as np
# 시드를 고정
np.random.seed(0)

# 발사대를 5대로 가정하여 진행
W = 5
# 위협 표적의 개수를 10개에서 100개까지 10개씩 증가시키며 실행
T = 10
# 표적 t에 할당할 수 있는 유도탄의 개수는 1개로 가정
FN_T = [1] * T

M = sum(FN_T)

# 0에서 4 사이의 랜덤한 값으로 구성된 W*T 행렬 생성 : 최초발사가능시점
ft_wt = np.random.rand(W, T) * 4

# 최소 3초 이상의 간격을 갖도록 설정
min_lt_wt = ft_wt + 3

# 발사가능시점으로부터 최소 3초 이상~ 전체 시간 10초 이내의 간격 행렬 생성
inter_lt_wt = 10 - min_lt_wt

# 3에서 10 사이의 랜덤한 값으로 구성된 W*T 행렬 생성 : 최후발사가능시점
lt_wt = np.random.rand(W, T) * inter_lt_wt + min_lt_wt

# 최초발사가능 시점과 최후발사가능 시점 사이의 간격이 3초 이상을 갖는지 확인
time_intervals_diff = lt_wt - ft_wt

# 0과 99 사이의 실수로 PK_wt 배열 초기화
PK_wt = np.random.uniform(0, 99, (W, T))

# 표적 t에 발사대 w를 할당할 때의 보상(요격확률) 행렬 생성
# PK_wt = np.random.rand(W, T)


In [57]:
# M을 W로 나누어 리스트에 적절히 분배
quotient, remainder = divmod(M, W)
m_w = [quotient] * W

# 나머지 값을 리스트의 앞부분에 추가
for i in range(remainder):
    m_w[i] += 1

print(m_w)


[2, 2, 2, 2, 2]


In [58]:
"""
목표함수 (1)
해당 목적함수는 결정변수 theta_wt와 요격확률의 곱의 합을 최대화함으로써 요격확률이 높을 때 표적-발사대 쌍의 발사 시점에 무기할당을 하는 의미로 해석할 수 있다.
"""
def objective_fun_1(theta_wt):
    res = 0
    for t in range(T):
        
        a = 1
        for w in range(W):
            for m in range(M):
                p = 1 - PK_wt[w,t] * theta_wt[w,t,m]
                a *= p
        
        #print(f"타깃 {t}에 대한 성공 값 : {1-a}")
        res += 1-a
    return res

In [59]:
import copy
# Initialize a 3D binary matrix of zeros
theta_wt = np.zeros((W, T, M), dtype=int)

# 미사일이 소모되는 리스트 left_FN_T으로 복사
left_FN_T = copy.copy(FN_T)

# 발사대에 적재된 유도탄 수량 리스트 left_m_w로 복사
left_m_w = copy.copy(m_w)

# 행렬의 원소들을 1차원 배열로 변환
flattened = PK_wt.flatten()

# 원소들을 내림차순으로 정렬
sorted_indices = np.argsort(flattened)[::-1]


i = 0
m = 0
indices_list = []
while True :
    # i번째 요격확률의 인덱스 v
    v = sorted_indices[i]
    
    row = v // T
    col = v % T
    print(f"Value: {flattened[v]}, Index: (Weapon : {row}, Target : {col})")
    
    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있다면,
    # 해당 Weapon의 적재된 유도탄이 남아 있다면,
    if left_FN_T[col] > 0 and left_m_w[row] > 0:
        # 할당할 유도탄 개수 감소
        left_FN_T[col] -= 1
        left_m_w[row] -= 1
        # 해당 W T M에 1 부여
        theta_wt[row][col][m] = 1
        # 무기가 할당된 indices_list에 추가
        indices_list.append(v)
        # 다음 미사일 인덱스로 이동
        m += 1

    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있지 않다면,
    elif left_FN_T[col] > 0:
        print(f"not enough Missile on Weapon {row}")
    
    else :
        print(f"not enough Missile for Target {col}")
    
    # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
    if sum(left_FN_T) == 0:
        break
    
    # 다음으로 요격확률이 높은 인덱스로 이동
    i += 1


Value: 98.88585365021879, Index: (Weapon : 4, Target : 9)
Value: 95.25666596662639, Index: (Weapon : 0, Target : 3)
Value: 94.65227983759917, Index: (Weapon : 2, Target : 2)
Value: 94.32215214018152, Index: (Weapon : 0, Target : 9)
not enough Missile for Target 9
Value: 91.02877876072269, Index: (Weapon : 4, Target : 7)
Value: 88.75811298925524, Index: (Weapon : 4, Target : 0)
not enough Missile on Weapon 4
Value: 88.30041214655154, Index: (Weapon : 4, Target : 3)
not enough Missile for Target 3
Value: 87.29180082363042, Index: (Weapon : 1, Target : 8)
Value: 87.229216514005, Index: (Weapon : 1, Target : 6)
Value: 83.79445857464165, Index: (Weapon : 1, Target : 1)
not enough Missile on Weapon 1
Value: 80.56598415054525, Index: (Weapon : 1, Target : 4)
not enough Missile on Weapon 1
Value: 79.8132049155625, Index: (Weapon : 4, Target : 4)
not enough Missile on Weapon 4
Value: 72.7842081901369, Index: (Weapon : 0, Target : 2)
not enough Missile for Target 2
Value: 71.8001737021444, Index

In [60]:
print(objective_fun_1(theta_wt))

767.7984881239656


In [61]:
indices = np.where(theta_wt == 1)


# Convert the indices to a list of tuples
indices_list = list(zip(*indices))

# Sort the indices by the second index (M)
indices_list.sort(key=lambda x: x[1])
print(f"목표 함수 값 : {objective_fun_1(theta_wt):.2f}")

for i, group in enumerate(indices_list):
    # Iterate over each group
    print(f"타깃 {group[1]}에 대하여 발사대 {group[0]}가 미사일을 발사합니다. 요격확률 : {PK_wt[group[0], group[1]]:.2f}")

목표 함수 값 : 767.80
타깃 0에 대하여 발사대 2가 미사일을 발사합니다. 요격확률 : 71.80
타깃 1에 대하여 발사대 0가 미사일을 발사합니다. 요격확률 : 26.73
타깃 2에 대하여 발사대 2가 미사일을 발사합니다. 요격확률 : 94.65
타깃 3에 대하여 발사대 0가 미사일을 발사합니다. 요격확률 : 95.26
타깃 4에 대하여 발사대 3가 미사일을 발사합니다. 요격확률 : 56.43
타깃 5에 대하여 발사대 3가 미사일을 발사합니다. 요격확률 : 58.50
타깃 6에 대하여 발사대 1가 미사일을 발사합니다. 요격확률 : 87.23
타깃 7에 대하여 발사대 4가 미사일을 발사합니다. 요격확률 : 91.03
타깃 8에 대하여 발사대 1가 미사일을 발사합니다. 요격확률 : 87.29
타깃 9에 대하여 발사대 4가 미사일을 발사합니다. 요격확률 : 98.89


In [62]:
"""
제약조건 (2)
수식 (2)는 동일 유도탄이 다수의 표적에 중복하여 할당되지 않기 위한 제약 조건이다.
"""

# 동일 유도탄이 다수의 표적에 중복 할당 X
def constraint2(theta_wt):
    W, T, M = theta_wt.shape
    for m in range(M):
        for w in range(W):
            # If the sum exceeds 1, immediately return False
            if np.sum(theta_wt[w,:,m]) > 1:
                return False
    # If none of the sums exceed 1, return True
    return True

# Now we can check the constraint:
print(constraint2(theta_wt))


True


In [63]:
"""
제약조건 (3)
각 위협 표적에 할당할 수 있는 유도탄의 개수를 제한하기 위하여 수식 (3)과 같은 제약식을 적용한다.
"""
# constraint3
def constraint3(theta_wt):
    W, T, M = theta_wt.shape
    for t in range(T):
        if np.sum(theta_wt[:,t,:]) > FN_T[t]:
            return False
    return True

print(constraint3(theta_wt))

True


In [64]:
"""
제약조건 (4)
한 발사대가 연속발사를 하기 위해서는 발사절차 및 발사대 안정화를 위한 시간을 고려하여야 하므로, 수식 (4)과 같이 한 발사대 상의 연속적 발사를 위한 최소 발사 간격을 제약으로 고려한다.
"""

def constraint4(theta_wt):
    return True

In [65]:
"""
제약조건 (5)
각 발사대에서 발사된 유도탄은 제한된 공간 영역 상에서 표적을 요격할 수 있다.
표적의 예상 궤적과 발사대 위치에 따른 요격 공간으로부터 교전 가능한 유도탄 발사 시간이 계산되어진다.
위협 표적에 대해 서 교전 가능한 발사 시간 구간 내에서 발사 시점이 산출되어야만 교전 가능하므로, 수식 (5)와 같이 표적 t에 대한 발사대 w에서 발사 가능한 시간에 대한 제약 조건을 고려해야 한다.
"""
def constraint5(theta_wt):
    return True

In [66]:
"""
제약조건 (6)
각 발사대는 적재된 수량만큼 유도탄을 사용할 수 있으므로, 수식 (6)과 같이 한 발사대에서 할당할 수 있는 유도탄의 개수는 적재량 이하여야 한다는 제약 조건을 고려한다.
"""
def constraint6(theta_wt):
    W, T, M = theta_wt.shape
    for w in range(W):
        if np.sum(theta_wt[w,:,:]) > m_w[w]:
            return False
    return True

print(constraint6(theta_wt))

True


타격 가능 시간 오름차순으로 정렬: 가장 먼저 타격 가능 시간이 빠른 타겟부터 발사할 계획을 세우면, 나중에 남는 시간을 더 유연하게 사용할 수 있습니다.

발사대 사용 계획: 발사대는 1초에 한 번씩만 발사가 가능하므로, 가능한 한 빠르게 그리고 빠르게 발사해야 하는 타겟을 먼저 선택합니다.

제한 시간 확인: 제한 시간이 10초라면, 최대 10번의 발사가 가능합니다. 따라서 가능한 한 빠른 시간 내에 많은 타겟을 처리해야 합니다.

동일 시간대의 타겟 처리: 만약 두 개 이상의 타겟이 동일한 시간에 발사해야 하는 경우, 그 중 하나는 놓치게 됩니다. 이러한 상황을 최소화하기 위해 가능한 한 빠른 시간대의 타겟을 우선적으로 처리합니다.

여기에 따라 간단한 알고리즘을 생각해볼 수 있습니다:

타겟의 '타격 가능 시간'을 오름차순으로 정렬합니다.

제한 시간 (10초) 내에 발사대가 발사할 수 있는 최대 횟수는 10번입니다.

정렬된 목록을 차례대로 확인하면서, 해당 타겟을 발사대가 처리할 수 있는지 확인합니다.

In [67]:
W, T, M = theta_wt.shape
for w in range(W):
    Target_Miassile_indices = np.where(theta_wt[w,:,:] == 1)

    Target_indices_list = Target_Miassile_indices[0]
    Miassile_indices_list = Target_Miassile_indices[1]

    print(Target_indices_list)
    ft_lt_w_t_list = []
    
    for w_t in Target_indices_list:
        ft_lt_w_t_list.append((ft_wt[w,w_t],lt_wt[w,w_t],w_t))
        # 최초발사가능시점,최후발사가능시점,t index
        sorted_ft_lt_w_t_list = sorted(ft_lt_w_t_list, key=lambda x: x[0])
        print(sorted_ft_lt_w_t_list)

[1 3]
[(2.860757465489678, 7.676235505713516, 1)]
[(2.1795327319875875, 5.671436402068977, 3), (2.860757465489678, 7.676235505713516, 1)]
[6 8]
[(0.08087358976130288, 8.76142952902932, 6)]
[(0.08087358976130288, 8.76142952902932, 6), (3.112627003799402, 9.370031409514059, 8)]
[0 2]
[(3.914473368931056, 9.927365052389222, 0)]
[(1.8459174490117274, 9.880224730137861, 2), (3.914473368931056, 9.927365052389222, 0)]
[4 5]
[(0.07515920174542057, 6.998784068185486, 4)]
[(0.07515920174542057, 6.998784068185486, 4), (2.4705419883035082, 6.672612544259232, 5)]
[7 9]
[(0.5157051906194132, 3.646088447992358, 7)]
[(0.5157051906194132, 3.646088447992358, 7), (1.4548430837704904, 4.480880236054584, 9)]
